In [ ]:
import pandas as pd
import category_encoders as ce

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [87]:
#Загрузка, анализ данных
train = pd.read_csv('/content/drive/MyDrive/Raif/data/train.csv', low_memory=False)
test = pd.read_csv('/content/drive/MyDrive/Raif/data/test.csv', low_memory=False)

In [2]:
train.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,0,0,0,0,2,4,6,3.293470,Пермь,1055397.0,0.027732,3,6,17,34,0,0,1,1,0,0,1,2,2,3,4,0,0,1,0,0,0,0,1,2,0,1,2,4,4,11,20,28,269.024986,3.368385,0,0,0,0.002864,4,13,21,139937.500000,136,49,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,0,0,1,0,2,2,6,43.950989,Орехово-Зуево,120184.0,0.089441,0,31,50,57,0,1,2,3,0,0,1,2,1,1,3,2,4,6,2,2,2,1,1,2,0,1,2,3,1,12,20,29,102.455451,1.476600,0,0,0,0.154661,4,10,11,60410.714286,146,37,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,0,1,1,0,3,6,11,2.676293,Ярославль,603961.0,0.200995,0,15,29,53,0,1,2,2,0,0,5,9,0,1,3,0,0,0,0,0,1,2,3,6,0,1,6,9,1,16,37,80,243.361937,1.455127,0,0,0,0.118275,9,13,21,45164.761264,105,27,1883.0,573.0,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,0,0,1,0,0,1,4,15.618563,Новокузнецк,552105.0,0.861400,0,0,0,5,0,0,0,0,0,0,0,1,0,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,4,11,295.633502,9.400855,0,0,0,1.036523,0,0,3,28805.263158,75,2,1801.0,54.0,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,1,1,1,0,8,14,26,9.995325,Химки,232066.0,0.236744,0,14,40,78,0,0,0,0,0,0,3,9,2,7,14,0,2,2,1,1,3,3,4,9,0,8,8,12,0,8,33,89,1.110429,1.235503,0,0,0,0.235032,10,32,62,13222.591362,144,38,3090.0,619.0,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0


In [ ]:
!pip install --upgrade category_encoders 

In [60]:
#Заполняем пропуски
train.reform_mean_floor_count_500 = train.reform_mean_floor_count_500.fillna(train.reform_mean_floor_count_500.median())
train.reform_mean_floor_count_1000 = train.reform_mean_floor_count_1000.fillna(train.reform_mean_floor_count_1000.median())
train.reform_mean_year_building_500 = train.reform_mean_year_building_500.fillna(train.reform_mean_year_building_500.mean())
train.reform_mean_year_building_1000 = train.reform_mean_year_building_1000.fillna(train.reform_mean_year_building_1000.mean())
train.reform_house_population_500 = train.reform_house_population_500.fillna(train.reform_house_population_500.median())
train.reform_house_population_1000 = train.reform_house_population_1000.fillna(train.reform_house_population_1000.median())
train.osm_city_nearest_population = train.osm_city_nearest_population.fillna(train.osm_city_nearest_population.mean())
train.street = train.street.fillna('missed')
#train = train.drop(['floor','id', 'region', 'osm_city_nearest_name', 'street', 'date'], 1)

test.reform_mean_floor_count_500 = test.reform_mean_floor_count_500.fillna(train.reform_mean_floor_count_500.median())
test.reform_mean_floor_count_1000 = test.reform_mean_floor_count_1000.fillna(train.reform_mean_floor_count_1000.median())
test.reform_mean_year_building_500 = test.reform_mean_year_building_500.fillna(train.reform_mean_year_building_500.mean())
test.reform_mean_year_building_1000 = test.reform_mean_year_building_1000.fillna(train.reform_mean_year_building_1000.mean())
test.reform_house_population_500 = test.reform_house_population_500.fillna(train.reform_house_population_500.median())
test.reform_house_population_1000 = test.reform_house_population_1000.fillna(train.reform_house_population_1000.median())
# test.street = test.street.fillna('missed')
#test.drop(['floor','id', 'region', 'street', 'osm_city_nearest_name', 'date'], axis=1, inplace=True)

In [6]:
train.shape

(279792, 77)

In [ ]:
train.reform_mean_year_building_1000[train.reform_mean_year_building_1000 == 1.0] = train.reform_mean_year_building_1000.mean()
train.reform_mean_year_building_500[train.reform_mean_year_building_500 == 1.0] = train.reform_mean_year_building_500.mean()

In [8]:
#Синтетическая метка по населению (5 групп)
cities = [
    'Москва', 'Санкт_петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
          'Нижниий Новгород', 'Челябинск', 'Самара', 'Омск', 'Ростов-на-Дону',
          'Уфа', 'Красноярск', 'Воронеж','Пермь', 'Волгоград', 'Краснодар',
          'Саратов', 'Тюмень', 'Тольятти', 'Ижевск', 'Барнаул', 'Ульяновск',
          'Иркутск', 'Хабаровск', 'Махачкала', 'Ярославль', 'Владивосток',
          'Оренбург', 'Томск', 'Кемерово', 'Новокузнецк', 'Рязань',
          'Набережные Челны', 'Астрахань', 'Киров', 'Пенза',
          'Севастополь', 'Балашиха', 'Липецк'
         ]
cities_mos = cities[0]
citiesi_spb = cities[1]
citiesi_mill = cities[2:15]
citiesi_half_mill = cities[16:]
train.city = train.city.apply(lambda x: 4 if x in cities_mos else 3 if x in citiesi_spb else 2 if x in citiesi_mill else 1 if x in citiesi_half_mill else 0)
test.city = test.city.apply(lambda x: 4 if x in cities_mos else 3 if x in citiesi_spb else 2 if x in citiesi_mill else 1 if x in citiesi_half_mill else 0)

In [9]:
train.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,2,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,0,0,0,0,2,4,6,3.293470,Пермь,1055397.0,0.027732,3,6,17,34,0,0,1,1,0,0,1,2,2,3,4,0,0,1,0,0,0,0,1,2,0,1,2,4,4,11,20,28,269.024986,3.368385,0,0,0,0.002864,4,13,21,139937.500000,136,49,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,0,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,0,0,1,0,2,2,6,43.950989,Орехово-Зуево,120184.0,0.089441,0,31,50,57,0,1,2,3,0,0,1,2,1,1,3,2,4,6,2,2,2,1,1,2,0,1,2,3,1,12,20,29,102.455451,1.476600,0,0,0,0.154661,4,10,11,60410.714286,146,37,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,1,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,0,1,1,0,3,6,11,2.676293,Ярославль,603961.0,0.200995,0,15,29,53,0,1,2,2,0,0,5,9,0,1,3,0,0,0,0,0,1,2,3,6,0,1,6,9,1,16,37,80,243.361937,1.455127,0,0,0,0.118275,9,13,21,45164.761264,105,27,1883.0,573.0,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,1,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,0,0,1,0,0,1,4,15.618563,Новокузнецк,552105.0,0.861400,0,0,0,5,0,0,0,0,0,0,0,1,0,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,4,11,295.633502,9.400855,0,0,0,1.036523,0,0,3,28805.263158,75,2,1801.0,54.0,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,4,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,1,1,1,0,8,14,26,9.995325,Химки,232066.0,0.236744,0,14,40,78,0,0,0,0,0,0,3,9,2,7,14,0,2,2,1,1,3,3,4,9,0,8,8,12,0,8,33,89,1.110429,1.235503,0,0,0,0.235032,10,32,62,13222.591362,144,38,3090.0,619.0,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0


In [10]:
train.shape

(279792, 77)

In [11]:
test.head()

,city,floor,id,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,lat,lng,total_square,street,date,realty_type,price_type
0,0,1.0,COL_289284,7,55,85,117,0,0,0,0,0,2,6,10,4.101661,Курск,443212.0,0.147072,0,7,7,19,0,0,0,0,1,2,3,3,5,5,5,2,2,4,7,7,8,0,0,1,0,0,0,1,5,36,61,83,435.363535,5.636811,0,0,0,0.108668,11,15,27,184,78,1997.0,743.0,4.325000,4.211268,1966.471591,1966.740260,Курская область,51.709255,36.147908,156.148996,S6983,2020-09-06,100,1
1,0,1.0,COL_289305,8,70,112,140,0,0,0,0,0,7,8,13,4.084249,Сургут,380632.0,0.162480,0,18,33,50,0,0,0,1,1,2,5,6,3,6,8,1,1,1,3,5,5,3,6,7,0,3,5,8,4,51,81,99,693.447931,9.641591,0,0,0,0.091416,9,11,19,118,44,2823.0,1019.0,5.389831,5.500000,1988.259259,1989.068182,Ханты-Мансийский АО,61.233240,73.462509,190.737943,S29120,2020-09-06,110,1
2,1,-1.0,COL_289318,3,28,67,122,0,0,0,0,0,9,18,34,1.291974,Тюмень,744554.0,0.075807,1,21,57,92,0,0,1,2,0,1,4,5,2,3,3,1,2,5,0,1,1,0,2,5,0,1,1,3,3,15,34,63,298.904686,1.803999,0,0,0,0.120308,6,20,29,150,51,3484.0,1332.0,7.915493,8.250000,1985.880282,1991.458333,Тюменская область,57.143110,65.554573,457.118051,S23731,2020-09-06,10,1
3,1,1.0,COL_289354,5,76,139,231,0,0,0,0,0,28,52,84,0.914598,Иркутск,623562.0,0.043604,1,30,60,95,1,8,13,18,0,5,6,10,5,8,12,12,20,32,1,8,18,1,5,5,0,7,11,16,3,14,26,51,60.224709,1.338052,0,0,0,0.079332,9,18,29,252,86,1975.0,666.0,3.276860,3.012048,1947.073276,1941.657895,Иркутская область,52.281380,104.282975,66.503622,S14207,2020-09-06,100,1
4,0,NaN,COL_289399,8,105,189,279,0,0,2,9,1,24,40,54,1.300135,Курск,443212.0,0.084204,1,40,72,84,1,10,12,12,3,10,14,21,2,6,6,10,14,17,1,1,1,4,5,12,0,2,12,16,1,49,98,156,432.530106,2.440972,0,0,0,0.093642,12,26,37,114,31,1044.0,394.0,4.346154,4.827586,1948.764151,1946.689655,Курская область,51.729706,36.194019,23.864915,S20658,2020-09-06,10,1


In [13]:
test.shape

(2974, 76)

In [14]:
# Делим на train и valid
train_null = train[train['price_type'] == 0]
valid_one = train[train['price_type'] == 1]

In [15]:
valid_one.shape

(4493, 77)

In [17]:
!pip install catboost

In [19]:
y_null = pd.DataFrame(train_null['per_square_meter_price'])
y_one = pd.DataFrame(valid_one['per_square_meter_price'])

In [20]:
train_null.shape

(275299, 77)

In [21]:
train_null = train_null.drop('per_square_meter_price', 1)
valid_one = valid_one.drop('per_square_meter_price', 1)

In [22]:
train_null.shape

(275299, 76)

In [23]:
valid_one.shape

(4493, 76)

In [24]:
y_null.shape

(275299, 1)

In [25]:
y_one.shape

(4493, 1)

In [26]:
# Проверка подключенного оборудования
from tensorflow.python.client import device_lib
def get_available_gpus():
  local_device_protos = device_lib.list_local_devices()
  print(local_device_protos)
  return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2530608324145603025
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11345264640
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17323822419570905926
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


['/device:GPU:0']

In [27]:
#CatBoostRegressor
from catboost import CatBoostRegressor

CatBoostRegressor
params = {'iterations': 6000,
          'task_type': 'GPU',
          'devices': '/device:GPU:0',
          'learning_rate': 0.001,
          'depth': 4,
          'l2_leaf_reg': 1,
          'eval_metric':'MAPE',
          'early_stopping_rounds': 200,
          'verbose': 200,
          'random_seed': 42}
         
cat = CatBoostRegressor(**params)

In [28]:
y_null.head()

,per_square_meter_price
0,139937.500000
1,60410.714286
2,45164.761264
3,28805.263158
4,13222.591362


In [29]:
train_null.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,2,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,0,0,0,0,2,4,6,3.293470,Пермь,1055397.0,0.027732,3,6,17,34,0,0,1,1,0,0,1,2,2,3,4,0,0,1,0,0,0,0,1,2,0,1,2,4,4,11,20,28,269.024986,3.368385,0,0,0,0.002864,4,13,21,136,49,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,0,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,0,0,1,0,2,2,6,43.950989,Орехово-Зуево,120184.0,0.089441,0,31,50,57,0,1,2,3,0,0,1,2,1,1,3,2,4,6,2,2,2,1,1,2,0,1,2,3,1,12,20,29,102.455451,1.476600,0,0,0,0.154661,4,10,11,146,37,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,1,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,0,1,1,0,3,6,11,2.676293,Ярославль,603961.0,0.200995,0,15,29,53,0,1,2,2,0,0,5,9,0,1,3,0,0,0,0,0,1,2,3,6,0,1,6,9,1,16,37,80,243.361937,1.455127,0,0,0,0.118275,9,13,21,105,27,1883.0,573.0,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,1,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,0,0,1,0,0,1,4,15.618563,Новокузнецк,552105.0,0.861400,0,0,0,5,0,0,0,0,0,0,0,1,0,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,4,11,295.633502,9.400855,0,0,0,1.036523,0,0,3,75,2,1801.0,54.0,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,4,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,1,1,1,0,8,14,26,9.995325,Химки,232066.0,0.236744,0,14,40,78,0,0,0,0,0,0,3,9,2,7,14,0,2,2,1,1,3,3,4,9,0,8,8,12,0,8,33,89,1.110429,1.235503,0,0,0,0.235032,10,32,62,144,38,3090.0,619.0,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0


In [30]:
train_null.city.values

array([2, 0, 1, ..., 1, 2, 2])

In [34]:
!pip install -U scikit-learn

In [35]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
import numpy as np

In [37]:
#TargetEncoder
cat_features = {'floor','id', 'region', 'street', 'osm_city_nearest_name', 'date', 'city'}

In [38]:
features = {'lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
       'osm_historic_points_in_0.005', 'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
       'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'region', 'total_square', 'street', 'date', 'realty_type',
       'price_type'}

In [ ]:
from category_encoders import TargetEncoder

train_null_TEncoded = TargetEncoder(cols=cat_features).fit_transform(train_null, y_null)
valid_one_TEncoded = TargetEncoder(cols=cat_features).fit_transform(valid_one, y_one)

In [44]:
# Переименовываем колонки 
for col in train_null_TEncoded[cat_features]:
    train_null_TEncoded.rename(columns={col: f'Encoded {col}'}, inplace=True)
    valid_one_TEncoded.rename(columns={col: f'Encoded {col}'}, inplace=True)

In [46]:
train_null_TEncoded

,Encoded city,Encoded floor,Encoded id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,Encoded osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,Encoded region,total_square,Encoded street,Encoded date,realty_type,price_type
0,62750.370602,88372.252662,110865.208853,57.998207,56.292797,4,19,35,52,0,0,0,0,0,2,4,6,3.293470,51652.600482,1055397.0,0.027732,3,6,17,34,0,0,1,1,0,0,1,2,2,3,4,0,0,1,0,0,0,0,1,2,0,1,2,4,4,11,20,28,269.024986,3.368385,0,0,0,0.002864,4,13,21,136,49,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,47746.364848,32.0,90954.708508,100043.051369,10,0
1,66471.302395,88372.252662,110865.208853,55.574284,39.543835,3,24,37,59,0,0,0,1,0,2,2,6,43.950989,42422.020437,120184.0,0.089441,0,31,50,57,0,1,2,3,0,0,1,2,1,1,3,2,4,6,2,2,2,1,1,2,0,1,2,3,1,12,20,29,102.455451,1.476600,0,0,0,0.154661,4,10,11,146,37,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,82131.731187,280.0,52212.945053,100043.051369,10,0
2,51439.192320,88372.252662,110865.208853,57.619140,39.850525,1,30,67,128,0,0,1,1,0,3,6,11,2.676293,50762.073001,603961.0,0.200995,0,15,29,53,0,1,2,2,0,0,5,9,0,1,3,0,0,0,0,0,1,2,3,6,0,1,6,9,1,16,37,80,243.361937,1.455127,0,0,0,0.118275,9,13,21,105,27,1883.0,573.0,6.141414,7.222222,1968.150000,1973.370370,48555.934069,297.4,63298.033270,100043.051369,110,0
3,51439.192320,88372.252662,110865.208853,53.897083,87.108604,0,0,5,21,0,0,0,1,0,0,1,4,15.618563,36054.599247,552105.0,0.861400,0,0,0,5,0,0,0,0,0,0,0,1,0,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,4,11,295.633502,9.400855,0,0,0,1.036523,0,0,3,75,2,1801.0,54.0,8.581081,9.000000,1992.716216,2014.000000,37166.528930,190.0,110865.208853,100043.051369,110,0
4,296852.620747,88372.252662,110865.208853,55.802590,37.487110,1,23,64,153,0,1,1,1,0,8,14,26,9.995325,211910.019877,232066.0,0.236744,0,14,40,78,0,0,0,0,0,0,3,9,2,7,14,0,2,2,1,1,3,3,4,9,0,8,8,12,0,8,33,89,1.110429,1.235503,0,0,0,0.235032,10,32,62,144,38,3090.0,619.0,7.263889,5.684211,1963.229167,1960.500000,296866.419584,60.2,58635.098626,100043.051369,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [55]:
train_null_TEncoded.drop('Encoded id', axis=1, inplace=True)

In [47]:
valid_one_TEncoded

,Encoded city,Encoded floor,Encoded id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,Encoded osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,Encoded region,total_square,Encoded street,Encoded date,realty_type,price_type
275299,61135.644015,64189.897858,69158.611507,56.063615,92.958428,0,7,14,26,0,0,0,0,0,0,0,2,7.795659,72113.919580,1095286.0,0.041323,2,6,25,35,0,0,0,2,0,1,2,3,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,5,10,15,1.087101,2.281425,0,0,0,0.052809,7,15,21,95,31,2670.0,940.0,7.136842,7.612903,1976.684211,1980.483871,72093.922350,106.351104,86696.240556,52201.168748,110,1
275300,57750.900239,64189.897858,69158.611507,51.534581,46.020549,13,198,345,462,0,0,0,0,4,44,66,89,0.987160,41496.134302,842097.0,0.144936,0,4,34,59,0,1,6,9,0,13,22,27,7,17,27,4,7,9,2,5,7,11,16,20,1,11,21,28,8,125,218,286,357.425239,1.924421,0,0,0,0.045451,22,34,45,502,104,4886.0,943.0,4.144608,4.164557,1940.591549,1939.348837,42591.745995,175.039602,47928.039179,52201.168748,10,1
275301,61135.644015,64189.897858,69158.611507,56.026884,92.818323,3,15,23,33,0,0,0,0,0,1,3,4,3.730568,72113.919580,1095286.0,0.141526,0,16,35,56,0,0,0,0,2,3,3,3,0,1,1,3,3,3,0,0,1,2,2,2,0,2,2,3,1,5,10,18,1.621353,0.579096,0,0,2,0.223787,6,11,18,175,66,4282.0,1445.0,6.851429,5.742424,1976.714286,1972.878788,72093.922350,89.534921,67778.662403,52201.168748,10,1
275302,57750.900239,64189.897858,69158.611507,52.275528,104.251444,0,10,26,40,0,1,1,1,0,2,6,8,2.344310,69211.524261,623562.0,0.281491,0,6,6,6,0,0,0,0,0,0,0,1,0,4,4,0,0,0,0,1,2,1,2,2,0,1,1,1,0,5,11,21,61.562546,0.813594,0,0,2,0.266067,8,11,19,230,56,1887.0,612.0,4.597222,5.469388,1968.098131,1975.530612,64213.498946,419.557752,43773.726982,52201.168748,10,1
275303,77016.339494,64189.897858,69158.611507,50.576545,36.584197,4,48,73,92,0,0,6,6,1,7,11,15,2.170184,47690.337549,384425.0,0.065996,1,19,28,46,1,5,10,11,0,2,2,4,3,3,4,2,2,2,0,2,2,10,11,15,0,0,0,0,1,19,29,36,555.411948,2.505330,0,0,0,0.092583,7,17,22,131,35,3837.0,1073.0,9.157480,9.088235,1983.125984,1980.470588,47096.483542,1098.703185,45336.495788,52201.168748,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [57]:
valid_one_TEncoded.drop('Encoded id', axis=1, inplace=True)

In [58]:
# Нормализация
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_null_norm = scaler.fit_transform(train_null_TEncoded)
valid_one_norm = scaler.fit_transform(valid_one_TEncoded)

In [59]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import numpy as np
catboost = CatBoostRegressor()

catboost_param_grid = {
    'loss_function': ['RMSE'],
    'eval_metric': ['RMSE'],
    'subsample': [0.35],
    'max_depth': [6],
    'n_estimators': [100],
    'reg_lambda': [65],
    'min_child_samples' : [20],
    'model_size_reg' : [0.1],
    'grow_policy' : ['Depthwise', 'SymmetricTree'],
    'random_strength' : [0.35],
    'bootstrap_type' : ['MVS', 'Bernoulli'],
    'bagging_temperature' : [50],
    'verbose': [1]
}

catboost_gs = GridSearchCV(
    estimator=catboost,
    param_grid=catboost_param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=4,
    n_jobs=-1,
    return_train_score=True
)

catboost_gs.fit(train_null_norm, np.log(y_null), eval_set=(valid_one_norm, np.log(y_one)))

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/catboost/core.py", line 5261, in fit
    save_snapshot, snapshot_file, snapshot_interval, init_model, callbacks, log_cout, log_cerr)
  File "/usr/local/lib/python3.7/dist-packages/catboost/core.py", line 1989, in _fit
    callbacks=callbacks
  File "/usr/local/lib/python

0:	learn: 1.2161963	test: 0.5733145	best: 0.5733145 (0)	total: 253ms	remaining: 25.1s
1:	learn: 1.1968175	test: 0.5582862	best: 0.5582862 (1)	total: 444ms	remaining: 21.7s
2:	learn: 1.1778198	test: 0.5446113	best: 0.5446113 (2)	total: 654ms	remaining: 21.1s
3:	learn: 1.1597256	test: 0.5315244	best: 0.5315244 (3)	total: 861ms	remaining: 20.7s
4:	learn: 1.1423692	test: 0.5204884	best: 0.5204884 (4)	total: 1.06s	remaining: 20.1s
5:	learn: 1.1257185	test: 0.5086901	best: 0.5086901 (5)	total: 1.26s	remaining: 19.8s
6:	learn: 1.1098594	test: 0.4973726	best: 0.4973726 (6)	total: 1.46s	remaining: 19.4s
7:	learn: 1.0947050	test: 0.4864632	best: 0.4864632 (7)	total: 1.65s	remaining: 19s
8:	learn: 1.0799077	test: 0.4770302	best: 0.4770302 (8)	total: 1.85s	remaining: 18.7s
9:	learn: 1.0656895	test: 0.4689178	best: 0.4689178 (9)	total: 2.06s	remaining: 18.5s
10:	learn: 1.0522547	test: 0.4598611	best: 0.4598611 (10)	total: 2.24s	remaining: 18.1s
11:	learn: 1.0391729	test: 0.4514641	best: 0.4514641 (

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f249bbf1990>,
             n_jobs=-1,
             param_grid={'bagging_temperature': [50],
                         'bootstrap_type': ['MVS', 'Bernoulli'],
                         'eval_metric': ['RMSE'],
                         'grow_policy': ['Depthwise', 'SymmetricTree'],
                         'loss_function': ['RMSE'], 'max_depth': [6],
                         'min_child_samples': [20], 'model_size_reg': [0.1],
                         'n_estimators': [100], 'random_strength': [0.35],
                         'reg_lambda': [65], 'subsample': [0.35],
                         'verbose': [1]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=4)

In [68]:
cat_pred_grid = catboost_gs.best_estimator_.predict(valid_one_norm)

In [70]:
print('RMSE', np.sqrt(mean_squared_error(np.log(y_one), cat_pred_grid)))

RMSE 0.3668708951436803


In [78]:
y_test = pd.read_csv('/content/drive/MyDrive/Raif/data/submission_cat_pred_grid_log.csv', low_memory=False)

In [80]:
y_test.head()

,id,per_square_meter_price
0,COL_289284,47046.182617
1,COL_289305,47573.834602
2,COL_289318,45092.396053
3,COL_289354,61161.174941
4,COL_289399,63948.302541


In [82]:
y_test = y_test.drop('id', 1)

In [83]:
test_TEncoded = TargetEncoder(cols=cat_features).fit_transform(test, y_test)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [84]:
cat_pred_grid_test = catboost_gs.predict(test_TEncoded)

In [86]:
# Запись результата
submission = pd.read_csv('/content/drive/MyDrive/Raif/data/test_submission.csv')
submission['per_square_meter_price'] = np.exp(cat_pred_grid_test)
submission.to_csv('submission_cat_pred_grid_log_hot.csv', index=False)